In [1]:
import functools, operator, requests, os, json
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import gradio as gr

In [2]:
# Set environment variables

import os
from dotenv import load_dotenv

load_dotenv("./../credentials_my.env")

os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ["AZURE_OPENAI_ENDPOINT_SCUS"]
os.environ["AZURE_OPENAI_API_KEY"]  = os.environ["AZURE_OPENAI_API_KEY_SCUS"]
os.environ["OPENAI_API_VERSION"]    = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["AZURE_OPENAI_API_TYPE"] = os.environ["OPENAI_API_TYPE"]

MODEL = os.environ["GPT4-0125PREVIEW-128k"]

# https://smith.langchain.com/
os.environ["LANGCHAIN_TRACING_V2"]  = "true"
os.environ["LANGCHAIN_PROJECT"]     = "LangGraph Research Agents"
os.environ["LANGCHAIN_API_KEY"]     = os.environ["LANGCHAIN_API_KEY"]

In [3]:
# Initialize model

llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0, max_tokens=1000)

In [4]:
# 1. Define custom tools
@tool("internet_search_tool", return_direct=False) # this is the name tracked in LangSmith
def internet_search_function(query: str) -> str:
    """Searches the internet using DuckDuckGo."""
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=5)]
        return results if results else "No results found."

@tool("process_content", return_direct=False)
def process_content_function(url: str) -> str:
    """Processes content from a webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

tools = [internet_search_function, process_content_function]

# AGENTS

## Create an Agent step by step

In [25]:
system_prompt = (
    "system",
    "You are a web searcher. Search the internet for information.")

mp1 = MessagesPlaceholder(variable_name="messages")

mp2 = MessagesPlaceholder(variable_name="agent_scratchpad")

print(system_prompt, mp1, mp2, sep='\n')

('system', 'You are a web searcher. Search the internet for information.')
variable_name='messages'
variable_name='agent_scratchpad'


In [16]:
cpt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        mp1,
        mp2
    ])

cpt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a web searcher. Search the internet for information.')), MessagesPlaceholder(variable_name='messages'), MessagesPlaceholder(variable_name='agent_scratchpad')])

In [12]:
agent = create_openai_tools_agent(llm, tools, cpt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a web searcher. Search the internet for information.')), MessagesPlaceholder(v

In [13]:
executor = AgentExecutor(agent=agent, tools=tools)
executor

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a web searcher. Search t

## Create an Agent Executor with a Helper Function

In [80]:
# Helper function for creating agents
def create_agent_executor(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [79]:
# Create the Search Agent Executor

search_agent_exec = create_agent_executor(
    llm=llm,
    tools=tools,
    system_prompt="You are a web searcher. Search the internet for information.",
    )

search_agent_exec

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a web searcher. Search t

In [ ]:
# Create the Search Agent Executor

insightresearch_agent_exec = create_agent_executor(
    llm           = llm,
    tools         = tools,
    system_prompt = "You are a web searcher. Search the internet for information.",
    )

insightresearch_agent_exec

# NODES

# GRAPH

## Create a Node for Researcher and Coder

In [ ]:
# This function facilitates creating nodes in the graph:
# it takes care of converting the agent response to a human message. 
# This is important because that is how we will add it the global state of the graph

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [28]:
from langchain_core.messages import HumanMessage

HumanMessage(content="This is the result", name="Node name")

HumanMessage(content='This is the result', name='Node name')

In [29]:
members = ["Researcher", "Coder"]
options = ["FINISH"] + members
options

['FINISH', 'Researcher', 'Coder']

## Create a Node for the Supervisor Agent

In [38]:
# Using openai function calling can make output parsing easier for us

function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

In [70]:
rb = llm.bind_functions(functions=[function_def], function_call="route")
rb

RunnableBinding(bound=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f9fc5a9aa90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f9fc5b10650>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=1000, azure_endpoint='https://mmopenaiscus.openai.azure.com/', deployment_name='gpt4-0125preview-128k', openai_api_version='2024-02-15-preview', openai_api_type='azure'), kwargs={'functions': [{'name': 'route', 'description': 'Select the next role.', 'parameters': {'title': 'routeSchema', 'type': 'object', 'properties': {'next': {'title': 'Next', 'anyOf': [{'enum': ['FINISH', 'Researcher', 'Coder']}]}}, 'required': ['next']}}], 'function_call': {'name': 'route'}})

### Partial Functions and functools.partial

A **partial function*** is a function that is created by fixing a certain number of arguments of an existing function. This is achieved using the functools.partial function, which allows you to set default values for one or more arguments of a function, effectively creating a new function with those default values already set.
The functools.partial function is used to create partial functions in Python. It allows you to fix a certain number of arguments of an existing function, effectively creating a new function with those fixed arguments. This can be useful when you want to create a simplified version of a function with some arguments pre-filled, or when you need to adapt a function to be used as a callback with a different signature.

By using **functools.partial**, you can create specialized functions from more general ones, making your code more modular and easier to maintain. This can be especially handy when working with libraries or frameworks that require functions with specific signatures.

In [71]:
system_prompt_supervisor = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

mp1 = MessagesPlaceholder(variable_name="messages")
mp2 = (
    "system",
    "Given the conversation above, who should act next?"
    " Or should we FINISH? Select one of: {options}")

cpt_supervisor = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_supervisor),
        mp1,
        mp2
    ]
).partial(options=str(options), members=", ".join(members))

cpt_supervisor

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'options': "['FINISH', 'Researcher', 'Coder']", 'members': 'Researcher, Coder'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['members'], template='You are a supervisor tasked with managing a conversation between the following workers:  {members}. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.')), MessagesPlaceholder(variable_name='messages'), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['options'], template='Given the conve

In [72]:
supervisor_chain = (
    cpt_supervisor
    | rb
    | JsonOutputFunctionsParser()
   
)

supervisor_chain

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'options': "['FINISH', 'Researcher', 'Coder']", 'members': 'Researcher, Coder'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['members'], template='You are a supervisor tasked with managing a conversation between the following workers:  {members}. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.')), MessagesPlaceholder(variable_name='messages'), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['options'], template='Given the conve

In [46]:
rb.invoke(supervisor_chain)

ValueError: Invalid input type <class 'langchain_core.runnables.base.RunnableSequence'>. Must be a PromptValue, str, or list of BaseMessages.

In [50]:
supervisor_chain | JsonOutputFunctionsParser()

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'options': "['FINISH', 'Researcher', 'Coder']", 'members': 'Researcher, Coder'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['members'], template='You are a supervisor tasked with managing a conversation between the following workers:  {members}. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.')), MessagesPlaceholder(variable_name='messages'), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['options'], template='Given the conve

In [63]:
double_by_triple_chain = (
    x
    | double
    | triple
)

NameError: name 'x' is not defined

# GRAPH!

In [73]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

NameError: name 'workflow' is not defined

In [6]:
[element["title"] for element in internet_search_function("What is a CLP?")]

["Commercial Learner's Permit - How To Get a CLP - CDLTraining.org",
 '7 Best Gun Oil, CLP, and Grease [Tested] - Pew Pew Tactical',
 "What is a Commercial Learner's Permit and how do you get one?",
 "What is a CLP? | How to Get a Commercial Learner's Permit | ACV Auctions",
 'CLP Prep Guide: How to Study for Your Permit Test']